In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

2023-10-20 02:03:07.282729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 02:03:07.943368: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-10-20 02:03:07.948631: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-10-20 02:03:08.274505: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


In [3]:
data = pd.read_csv('../dataset/A-Track-clean.csv')

In [4]:
patterns = data['payload'].tolist()
labels = data['label_action'].tolist()

In [5]:
#max_words = 10000
#maxlen = 100  # Sequence length
#tokenizer = Tokenizer(num_words=max_words)
#tokenizer.fit_on_texts(patterns)

#total_words = len(tokenizer.word_index) + 1

In [6]:
oov_tok = ""

 #토크나이저 설정
tokenizer = Tokenizer(oov_token=oov_tok, lower=False, split=' ')
tokenizer.fit_on_texts(patterns)

total_words = len(tokenizer.word_index) + 1


In [7]:
total_words

22883

In [7]:
input_sequences = tokenizer.texts_to_sequences(patterns)
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
unique_labels = list(set(labels))
num_classes = len(unique_labels)
label_to_index = {label: i for i, label in enumerate(unique_labels)}
labels = [label_to_index[label] for label in labels]
one_hot_labels = to_categorical(labels, num_classes=num_classes)
embedding_dim = 100

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense


model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len))
#model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(LSTM(32))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-10-19 11:01:37.450022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 11:01:37.450949: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_sequences, one_hot_labels, test_size=0.2, random_state=42)
classifier_nn = model.fit(X_train, y_train, epochs=10, verbose=True, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
563/563 [==============================] - 2848s 5s/step - loss: 0.8292 - accuracy: 0.6906 - val_loss: 0.5916 - val_accuracy: 0.7648
Epoch 2/10
143/563 [======>.......................] - ETA: 31:52 - loss: 0.5807 - accuracy: 0.7719